# Random Forests Classification

### Random Forests Briefly

Random forests fit an ensemble of regression or, in our case, classification trees over the domain of an outcome - here the attitude towards climate change associated with a tweet. 

Each tree estimator fits a piecewise constant surface over the outcome where the constant is the outcome's grand mean over the included observations. The tree is grown greedily by recursively splitting the observations in feature space at a cut-point in the feature dimension which maximally reduces the misclassification of the attitude outcome. Such trees are grown to maximum depth. 

In growing an individual tree, we overfit a high variance estimator to the data. Random forests reduce the variance of estimators by averaging over a large number of such trees.

A useful upshot of the random selection of features _before_ partitioning observations in feature space is that we are able to assess feature importance. 

NB The number `n_estimators` of trees averaged is not a real tuning parameter; as with the bootstrap, we need a sufficient number for the estimate to stabilize, but cannot overfit by having too many. In the hyperparameter tuning sections below, I include `n_estimators` in the grid search for convenience, to test if a sufficient number of trees were used. Our lowest cross-validated error is achieved by increasing the number of trees built. 

In [1]:
# Start Fresh

%reset -f

In [2]:
# Imports and Set Options

import csv  # for slang
import os
import re  # regex
import string  # punct
from pprint import pprint

import emoji  # for emoji
import gensim
import keras
import lightgbm as lgb
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from gensim.models import Word2Vec
from IPython.display import Image
from matplotlib import pyplot as plt
from nltk.corpus import stopwords  # stopwords
from nltk.stem import PorterStemmer  # stemming
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn import svm, tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier, RandomForestRegressor,
                              StackingClassifier)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import (accuracy_score, auc, average_precision_score,
                             brier_score_loss, classification_report,
                             confusion_matrix, f1_score, fbeta_score,
                             make_scorer, plot_precision_recall_curve,
                             precision_recall_curve, precision_score,
                             recall_score, roc_auc_score, roc_curve)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     cross_val_score, train_test_split)
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.svm import SVC  # "Support vector classifier"
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

Using TensorFlow backend.


## Homemade Classes and Functions

In [3]:
# Clean Text Class

class CleanText(BaseEstimator, TransformerMixin):
    
    def remove_mentions(self, input_text):
        '''
        Remove mentions, like @Mplamplampla
        '''
        return re.sub(r'@+', '', input_text)
    
    def remove_urls(self, input_text):
        '''
        Remove the urls mention in a tweet
        '''
        input_text  = ' '.join([w for w in input_text.split(' ') if '.com' not in w])
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        input_text = emoji.demojize(input_text)
        input_text = input_text.replace('_','')
        input_text = input_text.replace(':','')
        return input_text
    
    def possessive_pronouns(self, input_text):
        '''
        Remove the possesive pronouns, because otherwise after tokenization we will end up with a word and an s
        Example: government's --> ["government", "s"]
        '''
        return input_text.replace("'s", "")
    
    def characters(self, input_text):
        '''
        Remove special and redundant characters that may appear on a tweet and that don't really help in our analysis
        '''
        input_text = input_text.replace("\r", " ") # Carriage Return
        input_text = input_text.replace("\n", " ") # Newline
        input_text = " ".join(input_text.split()) # Double space
        input_text = input_text.replace('"', '') # Quotes
        return input_text
    
    def remove_punctuation(self, input_text):
        '''
        Remove punctuation and specifically these symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        '''
        punct = string.punctuation # string with all the punctuation symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        '''
        Remove numbers
        '''
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        '''
        Convert all the sentences(words) to lowercase
        '''
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        '''
        Remove stopwords (refers to the most common words in a language)
        '''
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        '''
        Reduce the words to their stem
        '''
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def encode_decode(self, input_text):
        '''
        Remove weird characters that are result of encoding problems
        '''
        return  " ".join([k.encode("ascii", "ignore").decode() for k in input_text.split(" ")])
    
    
    def translator(self, input_text):
        '''
        Transform abbrevations to normal words
        Example: asap --> as soon as possible
        '''
        input_text = input_text.split(" ")
        j = 0
        for _str in input_text:
            # File path which consists of Abbreviations.
            fileName = r"slang.txt"
            # File Access mode [Read Mode]
            accessMode = "r"
            with open(fileName, accessMode) as myCSVfile:
                # Reading file as CSV with delimiter as "=", so that 
                # abbreviation are stored in row[0] and phrases in row[1]
                dataFromFile = csv.reader(myCSVfile, delimiter="=")
                # Removing Special Characters.
                _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
                for row in dataFromFile:
                    # Check if selected word matches short forms[LHS] in text file.
                    if _str.upper() == row[0]:
                        # If match found replace it with its appropriate phrase in text file.
                        input_text[j] = row[1]
                myCSVfile.close()
            j = j + 1
        
        return(' '.join(input_text))
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords)
                    .apply(self.stemming))
        return clean_X
    
    def transform_no_stem(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords))
        return clean_X

## Read in Data and Create Train and Test Sets

In [4]:
# Read in data (Raw copy for reference; copy for processing)

tweets_raw = pd.read_csv('https://github.com/anilkeshwani/StatLearnProj/raw/master/Iason/climate_change_tweets_sample-2020-05-16-17-57.csv')
tweets = pd.read_csv('https://github.com/anilkeshwani/StatLearnProj/raw/master/Iason/climate_change_tweets_sample-2020-05-16-17-57.csv')
tweets.head()

,username,user_handle,date,retweets,favorites,text,label
0,WWF Climate & Energy,climateWWF,2020-04-28,11,22,Economic recovery and national climate pledges...,0
1,WWF Climate & Energy,climateWWF,2020-04-22,6,16,"In this difficult time, it’s hard to connect w...",0
2,WWF Climate & Energy,climateWWF,2020-04-01,43,69,"The decision to postpone # COP26, is unavoidab...",0
3,WWF Climate & Energy,climateWWF,2020-03-30,24,30,Japan - the world’s fifth largest emitter of g...,0
4,WWF Climate & Energy,climateWWF,2020-03-30,22,40,How can countries include # NatureBasedSolutio...,0


## Clean Dataset

Applies the class methods (leveraging `sklearn` API):

- translator
- remove_mentions
- remove_urls
- emoji_oneword
- possessive_pronouns
- remove_punctuation
- remove_digits
- encode_decode
- characters
- to_lower
- remove_stopwords
- stemming (via Porter Algorithm)

In [5]:
# Text Cleaning

# ct = CleanText()
# tweets["text"] = ct.fit_transform(tweets.text)
# tweets.to_csv("clean_tweets.csv") # save once processed
tweets = pd.read_csv("clean_tweets.csv") # read in instead
tweets = tweets.loc[(~tweets.text.isnull()), :]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(tweets.text, tweets.label, 
                                                    test_size=0.2, random_state=17, 
                                                    shuffle=True) # explicit default

[print(dat.head(3), dat.shape, end="\n"*2) for dat in [X_train, X_test, Y_train, Y_test]];

3642     might progress climat area not enough global s...
12695    trump crackdown politic scienc nasa climat div...
8451     no one would believ human panick fals climat m...
Name: text, dtype: object (14406,)

8376     nazi root environment climat chang fraud bbcne...
6111     interest democrat candid compar mani aspect cl...
13983    ittrademark imposs see global warm signal minn...
Name: text, dtype: object (3602,)

3642     0
12695    1
8451     1
Name: label, dtype: int64 (14406,)

8376     1
6111     0
13983    1
Name: label, dtype: int64 (3602,)



In [7]:
print(f"Training label counts: \n{Y_train.value_counts()}", end="\n"*2)
print(f"Test label counts: \n{Y_test.value_counts()}")

Training label counts: 
1    8433
0    5973
Name: label, dtype: int64

Test label counts: 
1    2138
0    1464
Name: label, dtype: int64


In [8]:
# Save set of workspace objects' names to enable periodic clean-up

necessities = set(dir())

## Word Vectorisations

In [9]:
tweets_raw.iloc[14156]["text"]

"CLIMATE CHANGE: Obama's right, in a way: if we change the political climate & get rid of progressives, we can help our national security!"

### Bag of Words (BOW) Binary ("One-Hot") Representation

In [10]:
# Bag of Words Representation (One Hot, i.e. binary)

BOW_vectorizer = CountVectorizer(stop_words = 'english', 
                                 binary=True, # Creates 0/1 "One Hot" vector; 
                                              # np.unique(BOW_train.toarray())
                                 min_df = 10)
BOW_vectorizer.fit(X_train)
BOW_train = BOW_vectorizer.transform(X_train)
BOW_test = BOW_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_BOW = MaxAbsScaler()
BOW_train_scaled = scaler_BOW.fit_transform(BOW_train)
BOW_test_scaled = scaler_BOW.transform(BOW_test)

In [11]:
# Most frequently occurring words in the training corpus

[(index, word) for index, word in sorted(BOW_vectorizer.vocabulary_.items(), key=lambda item: item[1], reverse=True)][:20]

[('zero', 2206),
 ('yr', 2205),
 ('youtub', 2204),
 ('youthvgov', 2203),
 ('youthtopow', 2202),
 ('youthclimatesummit', 2201),
 ('youth', 2200),
 ('young', 2199),
 ('york', 2198),
 ('yesterday', 2197),
 ('year', 2196),
 ('yeah', 2195),
 ('ye', 2194),
 ('yall', 2193),
 ('yale', 2192),
 ('ya', 2191),
 ('wwf', 2190),
 ('wsj', 2189),
 ('wrote', 2188),
 ('wrong', 2187)]

### Bag of Words with Frequencies Representation (FBOW)

In [12]:
# Bag of Words Representation (Frequencies; binary=False)

FBOW_vectorizer = CountVectorizer(stop_words = 'english', 
                                  binary=False, # Creates Word Frequency Vector; 
                                                # # np.unique(FBOW_train.toarray())
                                  min_df = 10)
FBOW_vectorizer.fit(X_train)
FBOW_train = FBOW_vectorizer.transform(X_train)
FBOW_test = FBOW_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_FBOW = MaxAbsScaler()
FBOW_train_scaled = scaler_FBOW.fit_transform(FBOW_train)
FBOW_test_scaled = scaler_FBOW.transform(FBOW_test)

In [13]:
# Word use (per tweet) frequencies

print(np.unique(FBOW_train.toarray(), return_counts=True))

# Feature_Index: Word Mapping

# {v: k for k, v in FBOW_vectorizer.vocabulary_.items()}

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 13], dtype=int64), array([31661370,   127070,     5161,      379,       45,        7,
              5,        1,        2,        1,        1], dtype=int64))


### Bag of Words Bigrams (bigram)

In [14]:
bigram_vectorizer = CountVectorizer(stop_words = 'english', 
                                    binary=True, 
                                    min_df = 10,
                                    ngram_range = (1,2)) # create bigrams
bigram_vectorizer.fit(X_train)

bigram_train = bigram_vectorizer.transform(X_train)
bigram_test = bigram_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_bigram = MaxAbsScaler()
bigram_train_scaled = scaler_bigram.fit_transform(bigram_train)
bigram_test_scaled = scaler_bigram.transform(bigram_test)

### Term Frequency–Inverse Document Frequency Representation (tf-idf)

In [15]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', 
                                   min_df=10) # used for now for consistency
tfidf_vectorizer.fit(X_train)
tfidf_train = tfidf_vectorizer.transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_tfidf = MaxAbsScaler()
tfidf_train_scaled = scaler_tfidf.fit_transform(tfidf_train)
tfidf_test_scaled = scaler_tfidf.transform(tfidf_test)

#### Reminder: Available objects produced by `fit`

Manual inspection of the attributes added after `fit`ting is useful; these are identified with a trailing underscore, e.g. `Cs_` or `scores_`

The `score_` attribute is a dictionary with one entry per outcome class; in our binary case, only the class `1` is stored in the dictionary. 

Each entry has dimension `(n_folds, n_cs, n_l1_ratios)`; i.e.  `(n_arrays, n_rows, n_cols)`. 

In [16]:
# objects produced by fit

# print("Fit Objects:")

# for item in dir(sklearn_Estimator):
#     if item.endswith("_") and (not item.startswith("_")):
#         print(item)

# Modelling

### Random Forests: BOW Representation

In [17]:
rf_clf = RandomForestClassifier(oob_score=True)

rf_clf.fit(BOW_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
rf_clf_cv_score = cross_val_score(rf_clf, BOW_train, Y_train)

print(f"Training Set Accuracy: {rf_clf.score(BOW_train, Y_train)}")
print(f"Out-of-Bag Score: {rf_clf.oob_score_}")

print(f"Cross-validated accuracy : {rf_clf_cv_score}") 
print(f"Mean CV accuracy : {np.round(np.mean(rf_clf_cv_score), 3)}")

print(f"Test set score : {rf_clf.score(BOW_test, Y_test)}")

Training Set Accuracy: 0.9991670137442732
Out-of-Bag Score: 0.8892822435096488
Cross-validated accuracy : [0.89417071 0.88441513 0.88719195 0.88996876 0.88024991]
Mean CV accuracy : 0.887
Test set score : 0.8903387007218212


#### Scaled Version

In [19]:
rf_clf_scaled = RandomForestClassifier(oob_score=True)

rf_clf_scaled.fit(BOW_train_scaled, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
rf_clf_scaled_cv_score = cross_val_score(rf_clf_scaled, BOW_train_scaled, Y_train)

print(f"Training Set Accuracy: {rf_clf_scaled.score(BOW_train_scaled, Y_train)}")
print(f"Out-of-Bag Score: {rf_clf_scaled.oob_score_}")

print(f"Cross-validated accuracy : {rf_clf_scaled_cv_score}") 
print(f"Mean CV accuracy : {np.round(np.mean(rf_clf_scaled_cv_score), 3)}")

print(f"Test set score : {rf_clf_scaled.score(BOW_test_scaled, Y_test)}")

Training Set Accuracy: 0.9991670137442732
Out-of-Bag Score: 0.8890045814244065
Cross-validated accuracy : [0.888966   0.88337383 0.88267963 0.88753905 0.88406803]
Mean CV accuracy : 0.885
Test set score : 0.8900610771793448


### Hyper-parameter Tuning via Grid Search

#### NB The below cell only appears once. The same hyperparameter grid and CV process is used across for all (subsequent) tuning.

In [21]:
# Set up Hyperparameter Search Space

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 100, None],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 3, 5],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [50, 100, 300]
}

# Set Cross-Validation Process

kfcv = KFold(n_splits=5, shuffle=True, random_state=101)

In [22]:
# Instantiate the grid search model
rf_grid_search = GridSearchCV(estimator = rf_clf, param_grid = param_grid, 
                              cv = kfcv, n_jobs = -1, verbose = 2)

In [23]:
rf_grid_search.fit(BOW_train, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed: 10.8min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_f...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, rando

In [27]:
# Best Score

print(f"Best Score: {rf_grid_search.best_score_}", end="\n"*2)

# Best Parameters

print("Best parameters:")
for k, v in rf_grid_search.best_params_.items():
    print(str(k) + ": " + str(v))

Best Score: 0.9021933166181745

Best parameters:
bootstrap: True
max_depth: None
max_features: log2
min_samples_leaf: 1
min_samples_split: 10
n_estimators: 300


### Random Forests: Bigrams Representation

In [28]:
rf_clf_bigram = RandomForestClassifier(oob_score=True)

rf_clf_bigram.fit(bigram_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
rf_clf_bigram_cv_score = cross_val_score(rf_clf_bigram, bigram_train, Y_train)

print(f"Training Set Accuracy: {rf_clf_bigram.score(bigram_train, Y_train)}")
print(f"Out-of-Bag Score: {rf_clf_bigram.oob_score_}")

print(f"Cross-validated accuracy : {rf_clf_bigram_cv_score}") 
print(f"Mean CV accuracy : {np.round(np.mean(rf_clf_bigram_cv_score), 3)}")

print(f"Test set score : {rf_clf_bigram.score(bigram_test, Y_test)}")

Training Set Accuracy: 0.9992364292655838
Out-of-Bag Score: 0.8939330834374566
Cross-validated accuracy : [0.89486468 0.88719195 0.89066296 0.89239847 0.88649774]
Mean CV accuracy : 0.89
Test set score : 0.8911715713492504


### Hyper-parameter Tuning via Grid Search

In [30]:
# Instantiate the grid search model
rf_grid_search_bigram = GridSearchCV(estimator = rf_clf_bigram, param_grid = param_grid, 
                                     cv = kfcv, n_jobs = -1, verbose = 2)

In [31]:
rf_grid_search_bigram.fit(bigram_train, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed: 10.1min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_f...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, rando

In [32]:
# Best Score

print(f"Best Score: {rf_grid_search_bigram.best_score_}", end="\n"*2)

# Best Parameters

print("Best parameters:")
for k, v in rf_grid_search_bigram.best_params_.items():
    print(str(k) + ": " + str(v))

Best Score: 0.9061502278321608

Best parameters:
bootstrap: True
max_depth: None
max_features: log2
min_samples_leaf: 1
min_samples_split: 10
n_estimators: 300


### Random Forests: FBOW Representation

In [33]:
rf_clf_FBOW = RandomForestClassifier(oob_score=True)

rf_clf_FBOW.fit(FBOW_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
rf_clf_FBOW_cv_score = cross_val_score(rf_clf_FBOW, FBOW_train, Y_train)

print(f"Training Set Accuracy: {rf_clf_FBOW.score(FBOW_train, Y_train)}")
print(f"Out-of-Bag Score: {rf_clf_FBOW.oob_score_}")

print(f"Cross-validated accuracy : {rf_clf_FBOW_cv_score}") 
print(f"Mean CV accuracy : {np.round(np.mean(rf_clf_FBOW_cv_score), 3)}")

print(f"Test set score : {rf_clf_FBOW.score(FBOW_test, Y_test)}")

Training Set Accuracy: 0.9992364292655838
Out-of-Bag Score: 0.8867138692211578
Cross-validated accuracy : [0.89243581 0.88441513 0.88510934 0.88684485 0.88129122]
Mean CV accuracy : 0.886
Test set score : 0.88811771238201


#### Scaled Version

In [35]:
rf_clf_FBOW_scaled = RandomForestClassifier(oob_score=True)

rf_clf_FBOW_scaled.fit(FBOW_train_scaled, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [36]:
rf_clf_FBOW_scaled_cv_score = cross_val_score(rf_clf_FBOW_scaled, FBOW_train_scaled, Y_train)

print(f"Training Set Accuracy: {rf_clf_FBOW_scaled.score(FBOW_train_scaled, Y_train)}")
print(f"Out-of-Bag Score: {rf_clf_FBOW_scaled.oob_score_}")

print(f"Cross-validated accuracy : {rf_clf_FBOW_scaled_cv_score}") 
print(f"Mean CV accuracy : {np.round(np.mean(rf_clf_FBOW_scaled_cv_score), 3)}")

print(f"Test set score : {rf_clf_FBOW_scaled.score(FBOW_test_scaled, Y_test)}")

Training Set Accuracy: 0.9992364292655838
Out-of-Bag Score: 0.8890045814244065
Cross-validated accuracy : [0.89174185 0.88684485 0.88545644 0.88858035 0.88337383]
Mean CV accuracy : 0.887
Test set score : 0.889505830094392


### Hyper-parameter Tuning via Grid Search

In [37]:
# Instantiate the grid search model
rf_grid_search_FBOW = GridSearchCV(estimator = rf_clf_FBOW, param_grid = param_grid, 
                                   cv = kfcv, n_jobs = -1, verbose = 2)

In [38]:
rf_grid_search_FBOW.fit(FBOW_train, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed:  9.8min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_f...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, rando

In [39]:
# Best Score

print(f"Best Score: {rf_grid_search_FBOW.best_score_}", end="\n"*2)

# Best Parameters

print("Best parameters:")
for k, v in rf_grid_search_FBOW.best_params_.items():
    print(str(k) + ": " + str(v))

Best Score: 0.9033042347611875

Best parameters:
bootstrap: True
max_depth: None
max_features: log2
min_samples_leaf: 1
min_samples_split: 10
n_estimators: 300


#### Hyperparameter Tuning for Scaled Version

In [40]:
# Instantiate the grid search model
rf_grid_search_FBOW_scaled = GridSearchCV(estimator = rf_clf_FBOW_scaled, param_grid = param_grid, 
                                          cv = kfcv, n_jobs = -1, verbose = 2)

In [41]:
rf_grid_search_FBOW_scaled.fit(FBOW_train_scaled, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed:  9.9min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_f...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, rando

In [42]:
# Best Score

print(f"Best Score: {rf_grid_search_FBOW_scaled.best_score_}", end="\n"*2)

# Best Parameters

print("Best parameters:")
for k, v in rf_grid_search_FBOW_scaled.best_params_.items():
    print(str(k) + ": " + str(v))

Best Score: 0.9024709498036986

Best parameters:
bootstrap: True
max_depth: None
max_features: log2
min_samples_leaf: 1
min_samples_split: 10
n_estimators: 300


### Random Forests: TF-IDF Representation

In [43]:
rf_clf_tfidf = RandomForestClassifier(oob_score=True)

rf_clf_tfidf.fit(tfidf_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [44]:
rf_clf_tfidf_cv_score = cross_val_score(rf_clf_tfidf, tfidf_train, Y_train)

print(f"Training Set Accuracy: {rf_clf_tfidf.score(tfidf_train, Y_train)}")
print(f"Out-of-Bag Score: {rf_clf_tfidf.oob_score_}")

print(f"Cross-validated accuracy : {rf_clf_tfidf_cv_score}") 
print(f"Mean CV accuracy : {np.round(np.mean(rf_clf_tfidf_cv_score), 3)}")

print(f"Test set score : {rf_clf_tfidf.score(tfidf_test, Y_test)}")

Training Set Accuracy: 0.9992364292655838
Out-of-Bag Score: 0.8848396501457726
Cross-validated accuracy : [0.88619015 0.87643179 0.88684485 0.88649774 0.88372093]
Mean CV accuracy : 0.884
Test set score : 0.8847862298722932


#### Scaled Version

In [45]:
rf_clf_tfidf_scaled = RandomForestClassifier(oob_score=True)

rf_clf_tfidf_scaled.fit(tfidf_train_scaled, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [46]:
rf_clf_tfidf_scaled_cv_score = cross_val_score(rf_clf_tfidf_scaled, tfidf_train_scaled, Y_train)

print(f"Training Set Accuracy: {rf_clf_tfidf_scaled.score(tfidf_train_scaled, Y_train)}")
print(f"Out-of-Bag Score: {rf_clf_tfidf_scaled.oob_score_}")

print(f"Cross-validated accuracy : {rf_clf_tfidf_scaled_cv_score}") 
print(f"Mean CV accuracy : {np.round(np.mean(rf_clf_tfidf_scaled_cv_score), 3)}")

print(f"Test set score : {rf_clf_tfidf_scaled.score(tfidf_test_scaled, Y_test)}")

Training Set Accuracy: 0.9992364292655838
Out-of-Bag Score: 0.8850478967097043
Cross-validated accuracy : [0.89000694 0.88129122 0.88337383 0.88441513 0.8781673 ]
Mean CV accuracy : 0.883
Test set score : 0.8831204886174348


### Hyper-parameter Tuning via Grid Search

In [47]:
# Instantiate the grid search model
rf_grid_search_tfidf = GridSearchCV(estimator = rf_clf_tfidf, param_grid = param_grid, 
                                    cv = kfcv, n_jobs = -1, verbose = 2)

In [48]:
rf_grid_search_tfidf.fit(tfidf_train, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed: 10.7min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_f...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, rando

In [49]:
# Best Score

print(f"Best Score: {rf_grid_search_tfidf.best_score_}", end="\n"*2)

# Best Parameters

print("Best parameters:")
for k, v in rf_grid_search_tfidf.best_params_.items():
    print(str(k) + ": " + str(v))

Best Score: 0.898791984913481

Best parameters:
bootstrap: True
max_depth: None
max_features: log2
min_samples_leaf: 1
min_samples_split: 2
n_estimators: 300


#### Hyperparameter Tuning for Scaled Version

In [50]:
# Instantiate the grid search model
rf_grid_search_tfidf_scaled = GridSearchCV(estimator = rf_clf_tfidf_scaled, param_grid = param_grid, 
                                           cv = kfcv, n_jobs = -1, verbose = 2)

In [51]:
rf_grid_search_tfidf_scaled.fit(tfidf_train_scaled, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed: 10.7min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_f...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, rando

In [52]:
# Best Score

print(f"Best Score: {rf_grid_search_tfidf_scaled.best_score_}", end="\n"*2)

# Best Parameters

print("Best parameters:")
for k, v in rf_grid_search_tfidf_scaled.best_params_.items():
    print(str(k) + ": " + str(v))

Best Score: 0.9003887972624973

Best parameters:
bootstrap: True
max_depth: None
max_features: log2
min_samples_leaf: 1
min_samples_split: 2
n_estimators: 300


In [90]:
sum(tweets_raw.label.value_counts())

18009

In [92]:
bigram_train[1]

<1x3201 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>